In [99]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [84]:
# dataframe to store extracted values
df = pd.DataFrame(columns=['id','name', 'summary', 'followers', 'employees_on_linkedin', 'overview', 'industry', 'company Size', 'headquarters', 'founded', 'specialties', 'location','website'])

# html files folder
folder_path = '' #folder containing html files


In [ ]:
dir_list = os.listdir(folder_path)
# loop through all the files in the folder
for filename in dir_list:
    print(filename)
    if filename.endswith('.html'):
        # load html into a soup obj
        with open(os.path.join(folder_path, filename), 'r',encoding="utf8") as f:
            soup = BeautifulSoup(f, 'html.parser')
            main_div = soup.select_one('.org-top-card__primary-content')
            
            name = ''
            if main_div:
                name = main_div.select_one('h1').text.strip()
                
            summary = soup.select_one('.org-top-card-summary__tagline')  
            if summary:
                summary = summary.text.strip()
            if main_div:
                top_card_location = main_div.select_one('div.org-top-card-summary-info-list__info-item:nth-of-type(1)')
                top_card_follower_count = main_div.select_one('div.org-top-card-summary-info-list__info-item:nth-of-type(2)')

                if top_card_location:
                    location = top_card_location.text.strip()

                if top_card_follower_count:
                    follower_count = top_card_follower_count.text.strip()
                    follower_count = ''.join(filter(str.isdigit, follower_count))

                #follower_count = main_div.select_one('div.org-top-card-summary-info-list__info-item:nth-of-type(3)').text.strip()
                employee_count = 0
                employee_count_content = soup.select_one('div.org-top-card-secondary-content__see-all-independent-link')

                if employee_count_content:
                    employee_anchor = employee_count_content.find('a')
                    print(employee_anchor)
                    if employee_anchor:
                        employee_span = employee_count_content.find('span')
                        if employee_span:
                            employee_count = employee_span.text.strip()
                            employee_count = ''.join(filter(str.isdigit, employee_count))
                            print(employee_count)
                    
            print(follower_count)
            # extract from overview
            overview_content = soup.select_one('.org-grid__content-height-enforcer')
            
            overview,website,industry,headquarters,founded,location = '','','','','',''
            company_size = 0
            
            #loop about us data list through content and add to a key value pair 
            if overview_content:
                overview_items = overview_content.find('dl')
                overview_dict = {}
                
                overview_items = overview_content.select_one('dl')
                if overview_items:
                    items = overview_items.select('dt, dd')
                    
                    for i in range(len(items)):
                        if items[i].name == 'dt':
                            key = items[i].text.strip()
                            value = ''
                            if i + 1 < len(items) and items[i + 1].name == 'dd':
                                value = items[i + 1].text.strip()
                            overview_dict[key] = value
                    print(overview_dict)
                
                #null and empty string checks
                if(overview_dict.__contains__('Industry')):
                    industry = overview_dict.get('Industry', '')  
                if(overview_dict.__contains__('Company size')):
                    company_size = overview_dict.get('Company size', '')  
                    #company_size = ''.join(filter(str.isdigit, company_size))
                if(overview_dict.__contains__('Headquarters')):
                    headquarters = overview_dict.get('Headquarters', '')  
                if(overview_dict.__contains__('Founded')):
                    founded = overview_dict.get('Founded', '')  
                if(overview_dict.__contains__('Specialties')):
                    specialties = overview_dict.get('Specialties', '')    
                if(overview_dict.__contains__('Website')):
                    website = overview_dict.get('Website', '')    
                
                location_card = soup.select_one('.org-location-card')
                if location_card:
                    primary_loc = location_card.find('p')
                    if primary_loc:
                        location = primary_loc.text.strip()
                        
                        
                org_grid_content = soup.find('div', {'class': 'org-grid__content-height-enforcer'})
                if org_grid_content:
                    section = org_grid_content.find('section')
                    overview_content = section.find('p')
                    if overview_content:
                        overview = overview_content.text.strip()
                    
                id = filename.replace(".html", "")    
                # append extracted data to dataframe
                df = df.append({'id':id,'name': name, 'summary': summary, 'followers': follower_count, 'employees_on_linkedin': employee_count, 'overview': overview, 'industry': industry, 'company Size': company_size, 'headquarters': headquarters, 'founded': founded, 'specialties': specialties, 'location': location,'website': website}, ignore_index=True)

In [92]:
temp = pd.read_excel('') #linked_in_link_scrape_v1.xlsx'

In [93]:
temp.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [ ]:
temp = temp[temp['linked_in_link'].notna()]

In [126]:
df['id'] = df['id'].astype(np.int64)
final_df = pd.merge(df,temp[['id','startup','website','linked_in_link']],on='id', how='left')


In [102]:
final_df.to_csv('linkedin_about_data_v1.csv', encoding='utf-8', index=False)

In [198]:
temp_final = final_df

In [199]:
import tldextract

def extract_domain(url):
    if(url):
        if isinstance(url, str):
            ext = tldextract.extract(url)
            return f"{ext.domain}.{ext.suffix}"
        else:
            return None
    else:
        return None
    
def ignore_subdomain_and_compare(url):
    if(url):
        if isinstance(url, str):
            ext = tldextract.extract(url)
            if ext.subdomain:
                domain = f"{ext.domain}.{ext.suffix}"
            else:
                domain = ext.registered_domain
            return domain.lower().split(".")[0]
        else:
            return None
    else:
        return None


temp_final['domain1'] = temp_final['website_x'].apply(ignore_subdomain_and_compare)
temp_final['domain2'] = temp_final['website_y'].apply(ignore_subdomain_and_compare)

temp_final['same_domain'] = (temp_final['domain1'] == temp_final['domain2']).astype(int)



In [200]:
def match_words(row):
    words1 = row['name'].split()
    words2 = row['startup'].split()
    
    return any(word.lower() in [w.lower() for w in words2] for word in words1)

temp_final['match_company_name'] = temp_final.apply(match_words, axis=1)

In [201]:
temp_final['company_or_domain_match'] = (temp_final['same_domain'] | temp_final['match_company_name']).astype(int)

In [202]:
temp_final['company_and_domain_match'] = (temp_final['same_domain'] & temp_final['match_company_name']).astype(int)

In [203]:
temp_final['is_greece'] = temp_final['location'].str.contains('GR').astype(int)

In [204]:
temp_final['is_greece_domain'] = temp_final['website_x'].str.contains('.gr').astype(int)

In [206]:
temp_final.to_csv('linkedin_about_data_w_domain_name_check_v1.csv', encoding='utf-8', index=False)

In [177]:
temp_final['company_and_domain_match'].value_counts()[1]

448